## Rapport : Résolution de Problèmes / Machine Learning

#### IMT Mines Alès 2022
##### INFRES 13 
- **Quentin BIALOTA**
- **Tom L'HERMENIER**

##### Imports
Veuillez lancer la cellule suivante pour importer les librairies utilisées dans ce rapport

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from IPython.display import display

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
### Import du dataset depuis OpenML (lien du dataset : https://www.openml.org/d/42737)

dataset = sklearn.datasets.fetch_openml(name="fps-in-video-games")

##### Structure du dataset :

Le data s

In [4]:
### Compte le nombre de NaN dans le dataset

bc_df = pd.DataFrame(dataset.data, columns=dataset.feature_names)

tab = []
j = 0

for feature in bc_df.columns:
    tab.append(bc_df[feature].isna().sum())

print(bc_df.isna().sum().sum())
print(np.c_[bc_df.columns,tab])

1299988
[['CpuName' 0]
 ['CpuNumberOfCores' 0]
 ['CpuNumberOfThreads' 0]
 ['CpuBaseClock' 0]
 ['CpuCacheL1' 0]
 ['CpuCacheL2' 0]
 ['CpuCacheL3' 5055]
 ['CpuDieSize' 202534]
 ['CpuFrequency' 0]
 ['CpuMultiplier' 0]
 ['CpuMultiplierUnlocked' 0]
 ['CpuProcessSize' 0]
 ['CpuTDP' 0]
 ['CpuNumberOfTransistors' 226039]
 ['CpuTurboClock' 0]
 ['GpuName' 0]
 ['GpuArchitecture' 0]
 ['GpuBandwidth' 15441]
 ['GpuBaseClock' 0]
 ['GpuBoostClock' 0]
 ['GpuBus nterface' 0]
 ['GpuNumberOfComputeUnits' 357107]
 ['GpuDieSize' 11196]
 ['GpuDirectX' 0]
 ['GpuNumberOfExecutionUnits' 412935]
 ['GpuFP32Performance' 56]
 ['GpuMemoryBus' 15441]
 ['GpuMemorySize' 15441]
 ['GpuMemoryType' 15441]
 ['GpuOpenCL' 177]
 ['GpuOpenGL' 0]
 ['GpuPixelRate' 0]
 ['GpuProcessSize' 0]
 ['GpuNumberOfROPs' 0]
 ['GpuShaderModel' 5]
 ['GpuNumberOfShadingUnits' 56]
 ['GpuNumberOfTMUs' 0]
 ['GpuTextureRate' 0]
 ['GpuNumberOfTransistors' 11539]
 ['GpuVulkan' 11525]
 ['GameName' 0]
 ['GameResolution' 0]
 ['GameSetting' 0]
 ['Dataset' 

##### Tri du dataset

Pour trier notre dataset, nous avons au à notre disposition deux méthodologies :
- Supprimer toutes les colonnes du dataset contenant des valeurs nulles (NaN), puis choisir dans les colonnes restantes, celles qui sont pertinentes.
- Supprimer la colonne `GpuNumberOfExecutionUnits` (car c'est une colonne contenant que des valeurs nulles), puis supprimer toutes les lignes contenant des valeurs nulles (NaN). Et comme pour la 1ere méthode, il faudra trier les colonnes pertinentes dans les colonnes restantes.

#### Suppression des colonnes contennants des valeurs nulles (NaN)

Dans notre dataset, certaines colonnes contiennent des valeurs non renseignées, nous avons donc décider de les supprimer.
Voici la liste :
- CpuCacheL3
- CpuDieSize
- CpuNumberOfTransistors
- GpuBandwidth
- GpuNumberOfComputeUnits
- GpuDieSize
- GpuNumberOfExecutionUnits
- GpuFP32Performance
- GpuMemoryBus
- GpuMemorySize
- GpuMemoryType
- GpuOpenCL
- GpuShaderModel
- GpuNumberOfShadingUnits
- GpuNumberOfTransistors
- GpuVulkan

In [5]:
dataset_df = pd.DataFrame(data=np.c_[dataset.data, dataset.target], columns=dataset.feature_names+['target'])

dataset_df_reduced = dataset_df.dropna(axis='columns')

dataset_df_reduced_rows = dataset_df.drop(
    columns=['GpuNumberOfExecutionUnits', 'GpuNumberOfComputeUnits','CpuNumberOfTransistors', 'CpuDieSize', 'CpuCacheL3','Dataset']).dropna()

dataset_df_reduced_rows = dataset_df_reduced_rows.drop_duplicates(subset=dataset_df_reduced_rows.columns.difference(['target'])).reset_index().drop(columns=['index'])

dataset_df_reduced_rows_pivot = dataset_df_reduced_rows.pivot_table(columns=['GameName'], aggfunc='size')

print("Nombre de jeux : ",dataset_df_reduced_rows_pivot)

%matplotlib inline

#print(dataset_df_reduced_rows.columns.size)

##pd.plotting.scatter_matrix(dataset.data, alpha=0.2)

#bc_df = pd.DataFrame(dataset.data, columns=dataset.feature_names)
#bc_df_target = pd.DataFrame(dataset.target, columns=['target'])

#df = pd.DataFrame(np.random.randn(1000, 4), columns=['A','B','C','D'])
#pd.plotting.scatter_matrix(df, alpha=0.2)

#_ = pd.plotting.scatter_matrix(
#                bc_df,
#                c=bc_df_target, 
#                figsize=(100,100), 
#                marker='o',
#                hist_kwds={'bins':20},s=60,
#                alpha=.8)


Nombre de jeux :  GameName
aWayOut                         1026
airMechStrike                   1026
apexLegends                     4771
arkSurvivalEvolved              1963
battlefield1                    4900
battlefield4                    3625
battletech                      1026
callOfDutyBlackOps4             2341
callOfDutyWW2                   1026
counterStrikeGlobalOffensive    9751
destiny2                        1026
dota2                           3801
farCry5                         1026
fortnite                        8394
frostpunk                       1026
grandTheftAuto5                 7911
leagueOfLegends                 6342
minecraft                       2958
overwatch                       7854
pathOfExile                     1026
playerUnknownsBattlegrounds     8859
radicalHeights                  1026
rainbowSixSiege                 1026
rocketLeague                    1607
rust                            2149
seaOfThieves                    1026
starcraft2 

In [ ]:
dataset_copy = dataset_df_reduced_rows
dict_init = {}
for name, values in dataset_df_reduced_rows.iteritems():
    compt = 0
    tab = {}
    for i in range(values.size):
        if isinstance(dataset_copy[name][i], str):
            if dataset_copy[name][i] not in tab:
                tab[dataset_copy[name][i]] = compt
                dataset_copy[name][i] = compt
                compt += 1
            else:
                dataset_copy[name][i] = tab[dataset_copy[name][i]]
        else:
            break
    print(len(tab))
    dict_init[name] = tab

In [ ]:
dataset_df_reduced_rows = dataset_df_reduced_rows.sample(random_state=0, n=dataset_df_reduced_rows.shape[0])
X = dataset_df_reduced_rows.drop(columns=['target'])
y = dataset_df_reduced_rows['target']

#print(X.info())

# Simple

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.fit_transform(X_test)

lr = LinearRegression().fit(X_train,y_train)

print("Training set score: {:.2f}".format(lr.score(X_train,y_train)))
print("Test set score: {:.2f}".format(lr.score(X_test,y_test)))

# plus complexe

pipeline = Pipeline([('transformer', scalar), ('estimator', LinearRegression())])

scores = cross_val_score(pipeline, X, y, cv=20)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
# print(lr.score(X_test, y_test))

pipeline = Pipeline([('transformer', scalar), ('estimator', Ridge(alpha=1.0))])

scores = cross_val_score(pipeline, X, y, cv=20)
print(scores)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

#clf = svm.SVC(kernel='linear', C=1, random_state=42)
#scores = cross_val_score(clf, X, y, cv=5)
#print(scores)